<a href="https://colab.research.google.com/github/rodrigoataidealves/TCC_MBA_USP/blob/main/02_Despesas_Visao_Siconfi_Bimestre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===========================
# Extração, leitura robusta e filtros
# ===========================

# from google.colab import files
# files.upload()  # se precisar fazer upload do ZIP

import os, zipfile, glob, pathlib, re, unicodedata
import pandas as pd

from google.colab import data_table, files
data_table.enable_dataframe_formatter()

ZIP_PATH = '/content/2025_1bi_mun_desp.zip'        # ajuste se necessário
EXTRACT_DIR = '/content/dados_2025_1bi_mun_desp'
OUT_XLSX = '/content/2025_1bi_mun_desp_filtrado.xlsx'

# 1) Extrair ZIP
os.makedirs(EXTRACT_DIR, exist_ok=True)
with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(EXTRACT_DIR)
print('Arquivos extraídos para:', EXTRACT_DIR)

# 2) Listar candidatos tabulares
candidates = []
for ext in ('*.csv','*.txt','*.xlsx','*.xls','*.parquet'):
    candidates.extend(glob.glob(os.path.join(EXTRACT_DIR, '**', ext), recursive=True))

if not candidates:
    raise FileNotFoundError('Nenhum arquivo CSV/TXT/Excel/Parquet encontrado no ZIP.')

print(f'{len(candidates)} arquivo(s) tabulares encontrado(s).')
for i, p in enumerate(candidates[:10], 1):
    print(f'[{i}]', p)
print('...')

# ------------------------
# Funções utilitárias
# ------------------------
def normalize_text(s:str) -> str:
    """normaliza texto: sem acento, minúsculo e só letras/números/espacos"""
    s = unicodedata.normalize('NFKD', str(s))
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    s = re.sub(r'[^a-z0-9]+', ' ', s)
    return s.strip()

def columns_norm_map(df: pd.DataFrame):
    """retorna dict: coluna_normalizada -> nome_original"""
    m = {}
    for c in df.columns:
        m[normalize_text(c)] = c
    return m

def try_read_csv_like(path):
    """Lê CSV/TXT tentando detectar separador, encoding e linha de cabeçalho."""
    # tentativas de encoding e linhas a pular (para ignorar capas/metadata)
    encodings = ['latin-1', 'utf-8', 'utf-8-sig']
    for enc in encodings:
        # tenta inferir o separador (sep=None + engine='python')
        for skip in range(0, 30):
            try:
                df = pd.read_csv(path, sep=None, engine='python', encoding=enc, skiprows=skip)
                # rejeita leitura trivial com 1 coluna e muitas linhas (sinal de separador errado)
                if df.shape[1] == 1 and df.shape[0] > 5:
                    continue
                return df
            except Exception:
                continue
    # fallback (pode vir 1 coluna; ainda assim retornaremos)
    try:
        return pd.read_csv(path, sep=None, engine='python', encoding='latin-1')
    except Exception:
        return pd.read_csv(path, encoding='latin-1')

def try_read_excel_like(path):
    """Lê Excel tentando encontrar a sheet e o offset de cabeçalho corretos."""
    xls = pd.ExcelFile(path)
    # percorre sheets e tenta pular de 0 a 30 linhas até achar header plausível
    for sheet in xls.sheet_names:
        for skip in range(0, 30):
            try:
                df = pd.read_excel(path, sheet_name=sheet, skiprows=skip)
                # descarta leituras vazias ou 1 coluna com muitas linhas (sheet de capa)
                if df is None or df.empty:
                    continue
                if df.shape[1] == 1 and df.shape[0] > 5:
                    continue
                return df, sheet, skip
            except Exception:
                continue
    # fallback: primeira sheet sem skip
    return pd.read_excel(path), xls.sheet_names[0], 0

def load_table(path):
    suf = pathlib.Path(path).suffix.lower()
    if suf in ('.csv', '.txt'):
        return try_read_csv_like(path), {'source': path, 'sheet': None, 'skiprows': None}
    elif suf in ('.xlsx', '.xls'):
        df, sheet, skip = try_read_excel_like(path)
        return df, {'source': path, 'sheet': sheet, 'skiprows': skip}
    elif suf == '.parquet':
        return pd.read_parquet(path), {'source': path, 'sheet': None, 'skiprows': None}
    else:
        raise ValueError(f'Formato não suportado: {suf}')

def find_col_by_candidates(df, candidates_norm:set):
    """Encontra a 1ª coluna cujo nome normalizado está no conjunto candidates_norm."""
    cmap = columns_norm_map(df)
    for norm_name, original in cmap.items():
        if norm_name in candidates_norm:
            return original
    raise KeyError(f"Nenhuma das colunas {sorted(list(candidates_norm))} foi encontrada.\n"
                   f"Colunas disponíveis: {list(df.columns)}")

# ------------------------
# 3) Abrir o MELHOR candidato
#     - tentaremos na ordem dos arquivos; se o 1º não tiver as colunas-alvo, testamos os demais
# ------------------------
targets_ok = False
chosen_info = None
for path in candidates:
    df_try, info = load_table(path)
    # normaliza cabeçalhos (sem renomear ainda)
    cmap = columns_norm_map(df_try)
    # Procuramos as colunas necessárias:
    #  - "coluna" (ex.: valores como "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d)")
    #  - "conta" OU "função"/"funcao" (muitos arquivos usam "Função")
    try:
        col_coluna = find_col_by_candidates(df_try, {'coluna'})
        col_conta  = find_col_by_candidates(df_try, {'conta', 'funcao', 'funcao orcamentaria', 'funcaoo', 'funcao classificacao', 'funcao despesa', 'funcao  '})
        # Se chegamos até aqui, este arquivo serve
        df = df_try
        chosen_info = info
        targets_ok = True
        break
    except KeyError:
        continue

if not targets_ok:
    # feedback para diagnóstico
    print("Nenhum arquivo com colunas 'coluna' e ('conta' ou 'função') foi encontrado diretamente.")
    print("Mostrando colunas do primeiro candidato para inspeção manual:")
    df_dbg, info_dbg = load_table(candidates[0])
    print(df_dbg.columns.tolist()[:50])
    raise KeyError("Não foi possível localizar as colunas necessárias. "
                   "Verifique se o dataset possui campos 'coluna' e 'conta' (ou 'função').")

print("Arquivo selecionado:", chosen_info['source'])
if chosen_info['sheet'] is not None:
    print("Planilha:", chosen_info['sheet'], "| skiprows:", chosen_info['skiprows'])
print("Dimensão bruta:", df.shape)

# ------------------------
# 4) Preparar filtros (case/acentos-agnósticos)
# ------------------------
def norm_series(s):
    return s.astype(str).map(normalize_text)

# localizar nomes originais novamente (garantia)
col_coluna = find_col_by_candidates(df, {'coluna'})
col_conta  = find_col_by_candidates(df, {'conta', 'funcao'})

# alvo da coluna "coluna"
alvo_coluna_norm = normalize_text('DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d)')

# valores aceitos na "Conta" (ou "Função")
contas_desejadas = ['Assistência Social', 'Previdência Social', 'Saúde', 'Educação', 'Encargos Especiais']
contas_norm_set  = { normalize_text(v) for v in contas_desejadas }

# cria colunas normalizadas temporárias para filtrar
col_coluna_norm = '__col_coluna_norm__'
col_conta_norm  = '__col_conta_norm__'

df[col_coluna_norm] = norm_series(df[col_coluna])
df[col_conta_norm]  = norm_series(df[col_conta])

mask = (df[col_coluna_norm] == alvo_coluna_norm) & (df[col_conta_norm].isin(contas_norm_set))
df_filtrado = df.loc[mask].drop(columns=[col_coluna_norm, col_conta_norm])

print("Registros após filtro:", df_filtrado.shape)

# Visualizar amostra
display(df_filtrado.head(50))

# ------------------------
# 5) Exportar Excel e disponibilizar download
# ------------------------
with pd.ExcelWriter(OUT_XLSX, engine='openpyxl') as xlw:
    df_filtrado.to_excel(xlw, index=False, sheet_name='filtrado')

print("Arquivo Excel salvo em:", OUT_XLSX)
files.download(OUT_XLSX)

Arquivos extraídos para: /content/dados_2025_1bi_mun_desp
1 arquivo(s) tabulares encontrado(s).
[1] /content/dados_2025_1bi_mun_desp/finbraRREO.csv
...
Arquivo selecionado: /content/dados_2025_1bi_mun_desp/finbraRREO.csv
Dimensão bruta: (2750317, 8)
Registros após filtro: (23481, 8)


,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor
380,Prefeitura Municipal de São Sebastião da Amore...,4126009,PR,8070,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Assistência Social,siconfi-cor_RREO2TotalDespesas,"248631,62"
383,Prefeitura Municipal de São Sebastião da Amore...,4126009,PR,8070,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Previdência Social,siconfi-cor_RREO2TotalDespesas,"85886,04"
385,Prefeitura Municipal de São Sebastião da Amore...,4126009,PR,8070,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Saúde,siconfi-cor_RREO2TotalDespesas,"2337545,18"
393,Prefeitura Municipal de São Sebastião da Amore...,4126009,PR,8070,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Educação,siconfi-cor_RREO2TotalDespesas,"2148879,72"
416,Prefeitura Municipal de São Sebastião da Amore...,4126009,PR,8070,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Encargos Especiais,siconfi-cor_RREO2TotalDespesas,"123747,8"
886,Prefeitura Municipal de Arroio do Tigre - RS,4301206,RS,12022,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Assistência Social,siconfi-cor_RREO2TotalDespesas,"211013,37"
889,Prefeitura Municipal de Arroio do Tigre - RS,4301206,RS,12022,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Saúde,siconfi-cor_RREO2TotalDespesas,"2117988,72"
895,Prefeitura Municipal de Arroio do Tigre - RS,4301206,RS,12022,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Educação,siconfi-cor_RREO2TotalDespesas,"2329540,75"
922,Prefeitura Municipal de Arroio do Tigre - RS,4301206,RS,12022,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Encargos Especiais,siconfi-cor_RREO2TotalDespesas,"507993,13"
1425,Prefeitura Municipal de Nova Roma do Sul - RS,4313359,RS,3468,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d),Assistência Social,siconfi-cor_RREO2TotalDespesas,"35393,33"


Arquivo Excel salvo em: /content/2025_1bi_mun_desp_filtrado.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>